# Using Elasticsearch Inference API along Hugging Face models

This notebook demonstrates how to use the Elasticsearch Inference API along with Hugging Face models to build a question and answer system. This notebook is based on the [Using Elasticsearch Inference API along Hugging Face models](https://www.elastic.co/search-labs/blog/elasticsearch-inference-api-and-hugging-face).

In [1]:
%pip install requests elasticsearch -q


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Installing dependencies and importing packages

In [32]:
import os
import json
import requests
import textwrap
import re
import time

from datetime import datetime
from dotenv import load_dotenv
from elasticsearch import Elasticsearch, helpers

load_dotenv()

True

## Setting up environment variables

Configure API keys and URLs for Elasticsearch and Hugging Face, along with the index name and inference endpoint identifier.

In [3]:
ELASTICSEARCH_API_KEY = os.getenv("ELASTICSEARCH_API_KEY")
ELASTICSEARCH_URL = os.getenv("ELASTICSEARCH_URL")
HUGGING_FACE_API_KEY = os.getenv("HUGGING_FACE_API_KEY")
HUGGING_FACE_INFERENCE_ENDPOINT_URL = os.getenv("HUGGING_FACE_INFERENCE_ENDPOINT_URL")


INDEX_NAME = "blog-posts"
INFERENCE_ENDPOINT_ID = "hugging-face-smollm3-3b"

## Elasticsearch Python client

Initialize the Elasticsearch client using the configured URL and API key.

In [4]:
es_client = Elasticsearch(ELASTICSEARCH_URL, api_key=ELASTICSEARCH_API_KEY)

## Hugging Face completions inference endpoint setup

Create an Elasticsearch inference endpoint that connects to the Hugging Face model for generating responses based on blog articles.

In [6]:
try:
    resp = es_client.inference.put(
        task_type="chat_completion",
        inference_id=INFERENCE_ENDPOINT_ID,
        body={
            "service": "hugging_face",
            "service_settings": {
                "api_key": HUGGING_FACE_API_KEY,
                "url": HUGGING_FACE_INFERENCE_ENDPOINT_URL,
            },
        },
    )

    print(
        "Chat completion inference endpoint created successfully:",
        resp["inference_id"],
    )
except Exception as e:
    print("Error creating chat completion inference endpoint:", {e})

Chat completion inference endpoint created successfully: hugging-face-smollm3-3b


### Creating index mapping

Define field types and properties for the blog articles index.

In [7]:
try:
    mapping = {
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "title": {
                    "type": "text",
                    "copy_to": "semantic_field",
                    "fields": {"keyword": {"type": "keyword"}},
                },
                "author": {"type": "keyword", "copy_to": "semantic_field"},
                "category": {"type": "keyword", "copy_to": "semantic_field"},
                "content": {"type": "text", "copy_to": "semantic_field"},
                "date": {"type": "date"},
                "semantic_field": {"type": "semantic_text"},
            }
        }
    }

    es_client.indices.create(index=INDEX_NAME, body=mapping)
    print(f"Index {INDEX_NAME} created successfully")
except Exception as e:
    print(f"Error creating index: {e}")

Error creating index: BadRequestError(400, 'resource_already_exists_exception', 'index [blog-posts/DRWJYsymSwuHPcvOFSM_DA] already exists')


In [37]:
def build_data(json_file, index_name):
    with open(json_file, "r") as f:
        data = json.load(f)

    for doc in data:
        action = {"_index": index_name, "_source": doc}
        yield action


try:
    success, failed = helpers.bulk(
        es_client,
        build_data("dataset.json", INDEX_NAME),
    )
    print(f"{success} documents indexed successfully")

    if failed:
        print(f"Errors: {failed}")
except Exception as e:
    print(f"Error: {str(e)}")

15 documents indexed successfully


## Semantic search function

Function to search for relevant articles using Elasticsearch semantic search capabilities.


In [27]:
def perform_semantic_search(query_text, index_name=INDEX_NAME, size=5):
    try:
        query = {
            "query": {
                "semantic": {
                    "field": "semantic_field",
                    "query": query_text,
                }
            },
            "size": size,
        }

        response = es_client.search(index=index_name, body=query)
        hits = response["hits"]["hits"]

        return hits
    except Exception as e:
        print(f"Semantic search error: {str(e)}")
        return []

In [70]:
def stream_chat_completion(messages: list, inference_id: str = INFERENCE_ENDPOINT_ID):
    url = f"{ELASTICSEARCH_URL}/_inference/chat_completion/{inference_id}/_stream"
    payload = {"messages": messages}
    headers = {
        "Authorization": f"ApiKey {ELASTICSEARCH_API_KEY}",
        "Content-Type": "application/json",
    }

    try:
        response = requests.post(url, json=payload, headers=headers, stream=True)
        response.raise_for_status()

        for line in response.iter_lines(decode_unicode=True):
            if line:
                line = line.strip()

                if line.startswith("event:"):
                    continue

                if line.startswith("data: "):
                    data_content = line[6:]

                    if not data_content.strip() or data_content.strip() == "[DONE]":
                        continue

                    try:
                        chunk_data = json.loads(data_content)

                        if "choices" in chunk_data and len(chunk_data["choices"]) > 0:
                            choice = chunk_data["choices"][0]
                            if "delta" in choice and "content" in choice["delta"]:
                                content = choice["delta"]["content"]
                                if content:
                                    yield content

                    except json.JSONDecodeError as json_err:
                        print(f"\nJSON decode error: {json_err}")
                        print(f"Problematic data: {data_content}")
                        continue

    except requests.exceptions.RequestException as e:
        yield f"Error: {str(e)}"

## Article recommendation system with latency tracking

SmolLM3-3B claims to deliver latencies between 40-200ms. We'll build a recommendation system that tracks latency at each stage to validate these claims while generating personalized article recommendations.

In [69]:
def recommend_articles(search_query, index_name=INDEX_NAME, max_articles=5):
    latencies = {}

    print(f"\n{'='*80}")
    print(f"🔍 Search Query: {search_query}")
    print(f"{'='*80}\n")

    articles = perform_semantic_search(search_query, index_name, size=max_articles)

    if not articles:
        print("❌ No relevant articles found.")
        return None, None, latencies

    print(f"✅ Found {len(articles)} relevant articles\n")

    # Build context with found articles
    context = "Available blog articles:\n\n"
    for i, article in enumerate(articles, 1):
        source = article.get("_source", article)
        context += f"Article {i}:\n"
        context += f"- Title: {source.get('title', 'N/A')}\n"
        context += f"- Author: {source.get('author', 'N/A')}\n"
        context += f"- Category: {source.get('category', 'N/A')}\n"
        context += f"- Date: {source.get('date', 'N/A')}\n"
        context += f"- Content: {source.get('content', 'N/A')}\n\n"

    # Simplified prompt that requests JSON in text
    # Simplified prompt that requests JSON in text
    system_prompt = """You are an expert content curator that recommends blog articles.

    Write recommendations in a conversational style starting with phrases like:
    - "If you're interested in [topic], this article..."
    - "This post complements your search with..."
    - "For those looking into [topic], this article provides..."

    Keep each recommendation concise (2-3 sentences max) and focused on VALUE to the reader.

    Return ONLY a valid JSON object with this exact structure:
    {
        "recommendations": [
            {"article_number": 1, "recommendation": "your recommendation text"},
            {"article_number": 2, "recommendation": "your recommendation text"},
            ... (continue for ALL articles)
        ]
    }"""

    user_prompt = f"""Search query: "{search_query}"

    {context}

    Generate a JSON object with one recommendation for EACH of the {len(articles)} articles above."""

    messages = [
        {"role": "system", "content": "/no_think"},
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    # LLM generation
    print(f"{'='*80}")
    print("🤖 Generating personalized recommendations...\n")

    llm_start = time.time()
    full_response = ""

    for chunk in stream_chat_completion(messages):  # Sin response_format
        print(chunk, end="", flush=True)
        full_response += chunk

    # Latency tracking
    llm_end = time.time()
    llm_latency = (llm_end - llm_start) * 1000
    latencies["llm_generation"] = llm_latency
    latencies["total"] = (llm_end - llm_start) * 1000

    print(
        f"\n\n{'='*80}\n📊 Latency Report:\n  • LLM generation: {latencies['llm_generation']:.2f}ms\n  • Total time: {latencies['total']:.2f}ms\n{'='*80}\n"
    )

    return articles, full_response, latencies

### Card visualization helper

Function to display recommendations in a card-style format for better visual differentiation.

In [71]:
def display_recommendation_cards(articles, recommendations_text):
    """
    Display article recommendations in a clean card format with title and personalized recommendation.
    """
    print("\n" + "=" * 100)
    print("📇 RECOMMENDED ARTICLES".center(100))
    print("=" * 100 + "\n")

    # Parse JSON recommendations - clean tags and extract JSON
    recommendations_list = []
    if recommendations_text:
        try:
            # Clean up <think> tags
            cleaned_text = re.sub(
                r"<think>.*?</think>", "", recommendations_text, flags=re.DOTALL
            )
            # Remove markdown code blocks ( ... ``` or ``` ... ```)
            cleaned_text = re.sub(r"```(?:json)?", "", cleaned_text)
            cleaned_text = cleaned_text.strip()

            parsed = json.loads(cleaned_text)
            recommendations_list = parsed.get("recommendations", [])
        except json.JSONDecodeError as e:
            print(f"⚠️  Could not parse recommendations as JSON: {e}")
            return

    for i, article in enumerate(articles, 1):
        source = article.get("_source", article)

        # Card border
        print("┌" + "─" * 98 + "┐")

        # Title
        title = source.get("title", "N/A")
        title_lines = textwrap.wrap(f"📌 {title}", width=94)
        for line in title_lines:
            print(f"│  {line}".ljust(99) + "│")

        # Card border
        print("├" + "─" * 98 + "┤")

        # Find recommendation for this article number
        recommendation = None
        for rec in recommendations_list:
            if rec.get("article_number") == i:
                recommendation = rec.get("recommendation")
                break

        # Only show if recommendation exists
        if recommendation:
            recommendation_lines = textwrap.wrap(recommendation, width=94)
            for line in recommendation_lines:
                print(f"│  {line}".ljust(99) + "│")

        # Card bottom
        print("└" + "─" * 98 + "┘")

## Usage example: Article recommendation system

Testing the recommendation system with Spanish articles to validate SmolLM3's multilingual capabilities and low latency performance.

In [72]:
search_query = "Security and vulnerabilities"

articles, recommendations, latencies = recommend_articles(search_query)

# Display visual cards
display_recommendation_cards(articles, recommendations)


🔍 Search Query: Security and vulnerabilities

✅ Found 5 relevant articles

🤖 Generating personalized recommendations...

<think>

</think>
```json
{
    "recommendations": [
        {
            "article_number": 1,
            "recommendation": "If you're interested in security and vulnerabilities, this article from the Security Team highlights a critical vulnerability in their authentication system. They recommend updating to SDK version 3.2.1 immediately as affected tokens have been automatically revoked."
        },
        {
            "article_number": 2,
            "recommendation": "This post complements your search with a detailed tutorial on migrating from version 1.x to 2.0, covering schema transformations and deprecated endpoints. It's essential for those planning the migration to do it in a staging environment first."
        },
        {
            "article_number": 3,
            "recommendation": "For those looking into updating to the new architecture, this articl

### Streaming function for real-time responses

Send messages to the Elasticsearch inference endpoint with streaming support, processing server-sent events to extract model responses in real-time.

`ask_question_streaming` function to put together the semantic search and the real time chat_completions.

## Use example

We'll ask about the articles that mention risks, vulnerabilities, or warnings.

## Cleanup

Delete the index and inference endpoints to prevent consuming resources after completing the workflow.

In [35]:
# Cleanup - Delete Index
es_client.indices.delete(index=INDEX_NAME)
print(f"Index {INDEX_NAME} deleted")

Index blog-posts deleted


In [ ]:
# Cleanup - Delete Inference Endpoint
es_client.inference.delete(inference_id=INFERENCE_ENDPOINT_ID)
print(f"Inference endpoint {INFERENCE_ENDPOINT_ID} deleted")

ObjectApiResponse({'acknowledged': True})